In [9]:
# hack to import from parent directory
import sys
import os.path
# get the current path
path = os.path.dirname(os.path.abspath(__name__))
# add the directory to the path
sys.path.insert(0, path)

from dqn_env_config import env_to_configs
# from maybe_wandb import get_wandb
# from dqn_helpers import dqn_cli, plot_and_save_average_plots

from ray.rllib.algorithms.dqn import DQNConfig, DQN
from ray.tune.logger import pretty_print
from ray import tune
from ray.tune.registry import register_env
import gymnasium as gym


In [10]:
chosen_env = "gym-pendulum"

config = env_to_configs[chosen_env]

class CustomEnv(gym.Env):
    def __init__(self, env_config):
        self.env = config.env_creator(config.action_space)
        self.action_space = self.env.action_space
        self.observation_space = self.env.observation_space

    def reset(self, seed, options):
        return self.env.reset()
    
    def step(self, action):
        return self.env.step(action)

register_env(chosen_env, lambda config: CustomEnv(config))


dqn_config = DQNConfig()

replay_config = {
    "type" : "ReplayBuffer",
    "capacity": config.memory_capacity
}

exploration_config={
    "type": "EpsilonGreedy", 
    "initial_epsilon": config.epsilon_init, 
    "final_epsilon": config.epsilon_min, 
    "epsilon_timesteps": config.epsilon_frac * config.max_steps * config.num_episodes,
}


dqn_config = dqn_config.training(
    replay_buffer_config=replay_config, 
    target_network_update_freq=config.target_update,
    grad_clip=config.gradient_clip,
    tau=config.tau,
    hiddens=config.hidden_layers,
)

dqn_config = dqn_config.env_runners(
    num_env_runners=1, 
    exploration_config=exploration_config,
    sample_batch_size=config.batch_size,
)
dqn_config = dqn_config.resources(num_gpus=1)
dqn_config = dqn_config.environment(env=chosen_env, )
algo = DQN(config=dqn_config)




for i in range(10):
    result = algo.train()
    print(pretty_print(result))

/home/unevilicorn/Desktop/base_implementation/venv/lib/python3.10/site-packages/ray/rllib/algorithms/algorithm.py:525: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/unevilicorn/Desktop/base_implementation/venv/lib/python3.10/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/home/unevilicorn/Desktop/base_implementation/venv/lib/python3.10/site-packages/ray/tune/logger/

agent_timesteps_total: 1000
counters:
  num_agent_steps_sampled: 1000
  num_agent_steps_trained: 0
  num_env_steps_sampled: 1000
  num_env_steps_trained: 0
custom_metrics: {}
date: 2024-06-08_15-57-51
done: false
env_runners:
  connector_metrics:
    ObsPreprocessorConnector_ms: 0.013866424560546875
    StateBufferConnector_ms: 0.005259513854980469
    ViewRequirementAgentConnector_ms: 0.12764930725097656
  custom_metrics: {}
  episode_len_mean: 200.0
  episode_media: {}
  episode_return_max: -872.3614916354418
  episode_return_mean: -1112.2394343517722
  episode_return_min: -1320.14075922966
  episode_reward_max: -872.3614916354418
  episode_reward_mean: -1112.2394343517722
  episode_reward_min: -1320.14075922966
  episodes_this_iter: 5
  episodes_timesteps_total: 1000
  hist_stats:
    episode_lengths: [200, 200, 200, 200, 200]
    episode_reward: [-1064.4182031452656, -1004.4914132021368, -1299.7853045463562,
      -872.3614916354418, -1320.14075922966]
  num_episodes: 5
  num_fault